In [1]:
#IMPORTS#
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
from collections import Counter
import string

In [4]:
#Peter Norvig's Spell Checker########################
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
################################################

In [5]:
filename = 'metamorphosis_clean.txt'
file = open(filename, 'rt')
text = file.read()
file.close()

In [6]:
len(text)

139058

In [110]:
clean_tokenized_text = []
stemmer = LancasterStemmer()
#stemmer = PorterStemmer() -- You may want to use Porter stemmer instead of Lancaster stemmer for higher efficiency, but lower accuracy


#tokenize, remove stop words, stemming, remove puncutation
stop_words = set(stopwords.words('english'))

tokens = word_tokenize(text)[:200] # tokenize
stripped = [w1.translate(str.maketrans('', '', string.punctuation)) for w1 in tokens] #remove punctuation
clean_words = [w2.lower() for w2 in stripped if (w2.isalpha() or w2.isdigit())] #check if it's char or a value
clean_words1 = [w3 for w3 in clean_words if not w3.lower() in stop_words] #remove stop words
clean_words2 = [stemmer.stem(w4) for w4 in clean_words1] #stemming (May not want to use it, up to you guys, may affect company values)
#clean_words3 = [correction(w3) for w3 in clean_words2] #I decided against using spell checker as it affected numerical values. It may also affect the company names. (e.g. Spotify -> Specify)
print("Original Length:\t",len(tokens))
print("Remove Punctuation:\t",len(stripped))
print("Check if char/number:\t",len(clean_words))
print("Remove StopWords:\t",len(clean_words1))
print("Stemming/Lemmatization:\t",len(clean_words2))

#From 200 words cut to 109


Original Length:	 200
Remove Punctuation:	 200
Check if char/number:	 165
Remove StopWords:	 109
Stemming/Lemmatization:	 109


==========START OF: Setting up for Twitter===========

In [2]:
import tweepy
from tweepy import OAuthHandler

In [3]:
# Consumer/Access key/secret/token obtained from Twitter
# You should have created a Twitter app and gotten these keys.
# Do NOT share your key/secret/token with other students.
consumer_key = 'xeXGwwXAkZsCLjXYlgTeQXqf9'
consumer_secret = 'b4BoFTR74ShjCAfyDWug5zmZ6UFNlntRmZOb6ZwbikErQe4Ut2'
access_token = '177596182-wt41IEWCf7yAYmscSYdMU24YSzdDsg7ctEO9RUw4'
access_secret = 'KQnLCPDQM5QjrbcaJR7gdkpdm09i8KIsGgIdDe0hS7rf5'


In [4]:
# The following two lines create an authorization object with your above authentication info.
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
# This line finally calls Twitter's Rest API.
api = tweepy.API(auth)

==========START OF: Gathering latest tweets with specific keywords===========

In [146]:
    
'''
watching now            containing both “watching” and “now”. This is the default operator.
“happy hour”            containing the exact phrase “happy hour”.
love OR hate            containing either “love” or “hate” (or both).
beer -root              containing “beer” but not “root”.
#haiku                  containing the hashtag “haiku”.
from:interior           sent from Twitter account “interior”.
to:NASA                 a Tweet authored in reply to Twitter account “NASA”.
@NASA                   mentioning Twitter account “NASA”.
politics filter:safe    containing “politics” with Tweets marked as potentially sensitive removed.
'''
tweetLimit = input("How many tweets to query?")
tweetSearch = input("What keywords do you want to query?")

How many tweets to query?10
What keywords do you want to query?Stocks


In [147]:
print("Searching for ",tweetLimit,"latest tweets with keyword(s):", tweetSearch)
print('===========================')
# This FOR loop will retrieve the latest X tweets - based on search term(s)

for tweet in tweepy.Cursor(api.search, q=tweetSearch).items(int(tweetLimit)):
    print('Tweet by: @' + tweet.user.screen_name + '  Tweet Msg:' +tweet.text)

Searching for  10 latest tweets with keyword(s): Stocks
Tweet by: @KBechNielsen  Tweet Msg:RT @KBechNielsen: Do you know the basic financial principles? 👉https://t.co/hYul4Wl4zl 👈

#financialfreedom #passiveincome #investing #dail…
Tweet by: @enggmrahman  Tweet Msg:Homebuilder stocks get a downgrade, as housing market flashes warning signs – MarketWatch https://t.co/RtKF6CFZBy https://t.co/WcWRfKxwlM
Tweet by: @Superdemo4  Tweet Msg:@Tancrede_Crptrs @MarionMarechal Comme toujours Collomb annonce quelque chose que l'on voit venir depuis plusieurs… https://t.co/y0JkMJJmWL
Tweet by: @KBechNielsen  Tweet Msg:RT @KBechNielsen: #Personalfinance explained 👉 https://t.co/hYul4Wl4zl 👈

#financialfreedom #passiveincome #keepgrowing #investing #dailymo…
Tweet by: @wallstreetstock  Tweet Msg:Stocks up the most today: $MGI, $SPWR, $GTHX, $KPFS and $LONE. https://t.co/JsaEM0f3iD
Tweet by: @HowardKoo_IM  Tweet Msg:New post: Digital Mining Market – Review Stocks Value https://t.co/tOKAyRlmFb
Tweet by:

==========START OF: Streaming latest tweets with specific keywords + Storage===========

In [148]:
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler

In [149]:
#Keeps the connection going and keep pulling new tweets from Twitter server.
#Writes the tweets into 'output.txt'. you need manually end the stream using the stop button at the top of the notebook
class MyListener(StreamListener):
    def on_data(self, data):
        try:
            with open('output.txt', 'a') as f:
                print(data)
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

    def on_error(self, status):
        print(status)
        return True

In [150]:
#Calls Twitter's Streaming API.
twitter_stream = Stream(auth, MyListener())

#Define our query terms in a list
query_terms = ['stock', 'boeing', 'apple', 'london', '#apple']
twitter_stream.filter(track=query_terms)

{"created_at":"Tue Jun 18 18:55:30 +0000 2019","id":1141056864416915463,"id_str":"1141056864416915463","text":"RT @DominiqueReynie: L\u2019avion europ\u00e9en de combat du futur est lanc\u00e9 au salon du Bourget par la France\ud83c\uddeb\ud83c\uddf7, l\u2019Allemagne\ud83c\udde9\ud83c\uddea et l\u2019Espagne\ud83c\uddea\ud83c\uddf8, quand\u2026","source":"\u003ca href=\"https:\/\/mobile.twitter.com\" rel=\"nofollow\"\u003eTwitter Web App\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2893109387,"id_str":"2893109387","name":"vivjg59","screen_name":"vivjg59","location":"FRANCE ","url":null,"description":"Directeur Financier de PME reconverti \u00e0 l'enseignement de l'Economie mais quelle id\u00e9e !","translator_type":"none","protected":false,"verified":false,"followers_count":18,"friends_count":8,"listed_count":1,"favourites_count":601,


{"created_at":"Tue Jun 18 18:55:31 +0000 2019","id":1141056865322844162,"id_str":"1141056865322844162","text":"RT @Nigel_Farage: Just filmed with @RealCandaceO for @prageru in London. She is fantastic. The episode should be out soon! https:\/\/t.co\/GmY\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/#!\/download\/ipad\" rel=\"nofollow\"\u003eTwitter for iPad\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":256471627,"id_str":"256471627","name":"David McGookin","screen_name":"DavidMcGookin","location":"belfast u.k.","url":null,"description":"Just a guy from the ghetto,Tigers bay, Belfast.Libertarian\/Brexiteer\/Unionist.Loath Socialism.Big fan of #LeedsUnited #Brantwoodfc #DallasCowboys #Southernrock","translator_type":"none","protected":false,"verified":false,"followers_count":1669,"friends_count":1769,"listed_count":37,"favourites

{"created_at":"Tue Jun 18 18:55:31 +0000 2019","id":1141056865712914432,"id_str":"1141056865712914432","text":"RT @TheRyanRagone: Yoooooo she was really at veterans stadium @AthleticsPISD @theestallion https:\/\/t.co\/ijWrj1Fbko","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1062934335349817344,"id_str":"1062934335349817344","name":"Megs \u2650\ufe0f","screen_name":"megsdeleonn","location":"Houston, TX","url":null,"description":"UHD\u201922 \ud83d\udc0a||CowboysNation \ud83c\udfc8\ud83d\udc99||J.P. \ud83d\udc97\ud83c\udf0e||","translator_type":"none","protected":false,"verified":false,"followers_count":155,"friends_count":414,"listed_count":0,"favourites_count":1552,"statuses_count":1047,"created_at":"Thu Nov 15 05:04:28 +0000 2018


{"created_at":"Tue Jun 18 18:55:31 +0000 2019","id":1141056867453546496,"id_str":"1141056867453546496","text":"RT @GeorgePapa19: America: do not forget that that current CIA director, Gina Haspel, was running the CIA desk in London in 2016 while Alex\u2026","source":"\u003ca href=\"https:\/\/mobile.twitter.com\" rel=\"nofollow\"\u003eTwitter Web App\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":729501868058120192,"id_str":"729501868058120192","name":"Marilyn Colon","screen_name":"Marilyn033Colon","location":"Avenel, NJ","url":null,"description":"Proud mom and devoted Christian. Ordinary.","translator_type":"none","protected":false,"verified":false,"followers_count":116,"friends_count":74,"listed_count":5,"favourites_count":14385,"statuses_count":919,"created_at":"Mon May 09 02:42:53 +0000 2016","utc_offset":null,"time_zone":null,"geo_e

{"created_at":"Tue Jun 18 18:55:31 +0000 2019","id":1141056868284080128,"id_str":"1141056868284080128","text":"RT @_Zeus_7: First training season for Sarri with Juventus \n\nSarri: Who's the fastest between you and Costa?\nRonaldo: Me \nSarri: Take this\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":200060522,"id_str":"200060522","name":"danpepperspray","screen_name":"danpepperspray","location":"Nigeria.","url":null,"description":"shot clock.","translator_type":"none","protected":false,"verified":false,"followers_count":2123,"friends_count":1542,"listed_count":4,"favourites_count":65,"statuses_count":63408,"created_at":"Fri Oct 08 10:42:17 +0000 2010","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":null,"contr

...

{"created_at":"Tue Jun 18 18:55:35 +0000 2019","id":1141056881651175424,"id_str":"1141056881651175424","text":"Finally decided to order some AirPods to be like Kwangmin and the box came today... but it was empty. Like Apple li\u2026 https:\/\/t.co\/n8uKUW80Bt","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1881318122,"id_str":"1881318122","name":"Andrew \ud83e\uddd1\ud83c\udffb","screen_name":"LostMemory1207","location":"California, USA","url":"https:\/\/lostmemory1207.carrd.co\/","description":"Once a BESTFRIEND Always a BESTFRIEND. Boyfriend fanboy account. \uc0ac\ub791\ud574\uc694 \ubbfc\uc6b0\ud615! Kim Donghyun replied to one of my tweets once. I also love Disneyland.","translator_type":"none","protected":false,"verified":false,"followers_count":

{"created_at":"Tue Jun 18 18:55:35 +0000 2019","id":1141056882896965632,"id_str":"1141056882896965632","text":"RT @justplanes: \u2708\ufe0f PLANE SPOTTING at NEW YORK JFK Airport\n\ud83d\udcfa WATCH FULL 30min VIDEO \ud83c\udf00 https:\/\/t.co\/lQh4gM1kbG   \n\n#avgeek #planespotting #jf\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":418980206,"id_str":"418980206","name":"Antonio M Pereira\ud83d\ude81\ud83d\udee9\ufe0f\ud83c\udde7\ud83c\uddf7","screen_name":"Pereiraosv","location":"Brazil","url":"http:\/\/www.ampairsafety.com.br","description":"Director of AMP Air Safety Consulting - Air Safety Investigator - Safety Management System Consultant - Risk Management - http:\/\/ampairsafety.com.br","translator_type":"none","prot

{"created_at":"Tue Jun 18 18:55:35 +0000 2019","id":1141056884251566081,"id_str":"1141056884251566081","text":"@milkxheese you are the apple of my eyes,\ngatau kenapa aku jatuh cinta bgt sama ni film karna alurnya keren bgt si,\u2026 https:\/\/t.co\/undYgg5eSh","display_text_range":[12,140],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":1141028283917541376,"in_reply_to_status_id_str":"1141028283917541376","in_reply_to_user_id":1041576819579736064,"in_reply_to_user_id_str":"1041576819579736064","in_reply_to_screen_name":"milkxheese","user":{"id":905059410367586304,"id_str":"905059410367586304","name":"aku adalah","screen_name":"sail0rmoon_","location":null,"url":null,"description":null,"translator_type":"none","protected":false,"verified":false,"followers_count":12,"friends_count":94,"listed_count":0,"favourites_count":546,"statuses_count":377,"created_at":"Tue Sep 05 1

{"created_at":"Tue Jun 18 18:55:35 +0000 2019","id":1141056885312933895,"id_str":"1141056885312933895","text":"RT @TechLondonAdv: EU-funded research names London as the best city in the continent for developing tech for good - from profit tech compan\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":896663613477408768,"id_str":"896663613477408768","name":"Discovering Technology","screen_name":"discoveringtech","location":null,"url":"https:\/\/discoveringtechnology.co","description":"Helping businesses discover new tech ideas & potential solutions to challenges they face... For more info visit our website \ud83d\udc47#tech #innovation #ideationevents","translator_type":"none","protected":false,"verified":false,"followers_count"

KeyboardInterrupt: 

==========START OF: Extracting tweets from a particular user===========<br />
**Twitter allows access to only 3240 tweets via this method

In [18]:
import tweepy
from tweepy import OAuthHandler
import csv

In [15]:
userToScrape = input("Enter the twitter handle of the person whose tweets you want to download:- ")

Enter the twitter handle of the person whose tweets you want to download:- Michael_OMG


In [19]:
print("Starting to scrape @" + userToScrape)
# initialization of a list to hold all Tweets
all_the_tweets = []

# We will get the tweets with multiple requests of 200 tweets each
new_tweets = api.user_timeline(screen_name=userToScrape, count=200)

# saving the most recent tweets
all_the_tweets.extend(new_tweets)

# save id of 1 less than the oldest tweet
oldest_tweet = all_the_tweets[-1].id - 1

# grabbing tweets till none are left
while len(new_tweets) > 0:
# The max_id param will be used subsequently to prevent duplicates
    new_tweets = api.user_timeline(screen_name=userToScrape, count=200, max_id=oldest_tweet)

    # save most recent tweets
    all_the_tweets.extend(new_tweets)

    # id is updated to oldest tweet - 1 to keep track
    oldest_tweet = all_the_tweets[-1].id - 1
    print ('...%s tweets have been downloaded so far' % len(all_the_tweets))

    # transforming the tweets into a 2D array that will be used to populate the csv
    outtweets = [[tweet.id_str, tweet.created_at,
    tweet.text.encode('utf-8')] for tweet in all_the_tweets]

    # writing to the csv file
    with open(userToScrape + '_tweets.csv', 'w', encoding='utf8') as f:
        writer = csv.writer(f)
        writer.writerow(['id', 'created_at', 'text'])
        writer.writerows(outtweets)

Starting to scrape @Michael_OMG
...400 tweets have been downloaded so far
...600 tweets have been downloaded so far
...800 tweets have been downloaded so far
...1000 tweets have been downloaded so far
...1200 tweets have been downloaded so far
...1396 tweets have been downloaded so far
...1596 tweets have been downloaded so far
...1796 tweets have been downloaded so far
...1996 tweets have been downloaded so far
...2196 tweets have been downloaded so far
...2396 tweets have been downloaded so far
...2596 tweets have been downloaded so far
...2796 tweets have been downloaded so far
...2896 tweets have been downloaded so far
...2896 tweets have been downloaded so far
